In [1]:
import os, sys, time, glob
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pandana as pdna
pd.options.mode.chained_assignment = None
import h5py
sys.path.append(os.path.join(os.getcwd(),r'..\..\..'))
from input_configuration import base_year

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
output_dir = r'..\..\..\outputs\compare\RTP\mode_share'

# Mode Share

In [5]:
pd.options.display.float_format = '{:0,.1%}'.format
df_trip = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_county.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

### Region

In [6]:
result_df = pd.DataFrame()
df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'
for purp in ['All Trips','Work','Non-Work']:
    if purp != 'All Trips':
        df = df_trip[df_trip['trip_type'] == purp]
    else:
        df = df_trip.copy()
    _df = df.groupby('mode').sum()[['trexpfac']]/df['trexpfac'].sum()
    _df['purp'] = purp
    result_df = result_df.append(_df)
_df = result_df.pivot_table(columns='purp', index='mode', values='trexpfac', aggfunc='sum')
_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_region.csv')); incr+=1
_df = _df.reset_index()
_df.rename(columns={'mode': 'Mode'}, inplace=True)
_df = _df.rename_axis(None, axis=1)
HTML(_df.to_html(index=False))

Mode,All Trips,Non-Work,Work
Bike,1.4%,1.4%,1.2%
HOV2,22.9%,25.1%,10.4%
HOV3+,15.8%,17.7%,4.8%
SOV,41.5%,37.4%,64.9%
School Bus,2.0%,2.4%,0.0%
TNC,1.0%,0.8%,2.6%
Transit,2.9%,2.4%,6.3%
Walk,12.4%,12.8%,9.9%


### County
All Trips

In [7]:
def calculate_mode_share(df, geography, purp=None):
    
    df.loc[df['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
    df.loc[df['dpurp'] == 'Work', 'trip_type'] = 'Work'
    
    if purp:
        df = df[df['trip_type'] == purp]
    
    _df = df.pivot_table(index='mode', columns=geography, values='trexpfac', aggfunc='sum')
    
    for col in _df:
        _df[col] = _df[col]/_df[col].sum()
    return _df.T

In [8]:
df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_county.csv')
df = calculate_mode_share(df, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_county.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_county': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
King,1.5%,21.7%,14.7%,40.5%,1.9%,1.2%,4.1%,14.3%
Kitsap,1.2%,23.8%,15.6%,42.5%,1.8%,0.8%,2.1%,12.2%
Pierce,1.4%,24.8%,17.4%,41.9%,2.3%,0.8%,1.5%,10.1%
Snohomish,1.2%,24.1%,17.3%,43.7%,2.2%,0.8%,1.3%,9.4%


Non-Work Trips

In [9]:
df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_county.csv')
df = calculate_mode_share(df, 'hh_county', 'Non-Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_county_nonwork.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_county': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
King,1.5%,24.0%,16.7%,36.6%,2.2%,0.9%,3.5%,14.6%
Kitsap,1.2%,25.9%,17.2%,38.8%,2.1%,0.6%,1.4%,12.8%
Pierce,1.4%,26.8%,19.2%,37.6%,2.6%,0.6%,1.0%,10.7%
Snohomish,1.3%,26.2%,19.3%,39.0%,2.5%,0.6%,0.9%,10.1%


Work Trips

In [10]:
df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_county.csv')
df = calculate_mode_share(df, 'hh_county', 'Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_county_work.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_county': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
King,1.4%,9.9%,4.6%,60.8%,0.0%,3.0%,7.6%,12.7%
Kitsap,1.1%,10.4%,4.9%,66.9%,0.0%,2.0%,6.3%,8.4%
Pierce,1.0%,11.0%,5.2%,70.8%,0.0%,1.8%,4.5%,5.7%
Snohomish,1.0%,11.1%,5.3%,72.0%,0.0%,2.0%,3.5%,5.1%


### Regional Growth Center

All Trips

In [11]:
df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_rgc.csv')

# Create results that sum up trips for households in all RGCs
_df = df[df['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
_df['hh_rgc'] = 'All RGCs'
df = df.append(_df)
df = calculate_mode_share(df, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rgc.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rgc': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
All RGCs,2.7%,13.1%,7.0%,26.2%,0.8%,2.1%,7.0%,41.1%
Auburn,1.6%,21.4%,13.9%,32.2%,1.5%,1.1%,6.7%,21.6%
Ballard-Interbay,3.1%,16.2%,6.8%,42.2%,0.2%,1.6%,7.0%,22.9%
Bellevue,1.6%,12.9%,6.6%,26.4%,0.7%,1.7%,4.2%,45.8%
Bothell Canyon Park,2.5%,20.5%,14.9%,47.7%,1.2%,1.0%,1.7%,10.6%
Bremerton,2.3%,16.7%,9.9%,29.6%,1.3%,1.2%,2.4%,36.6%
Burien,1.8%,20.6%,12.4%,40.5%,2.1%,1.0%,5.7%,15.8%
Cascade,1.4%,25.8%,19.0%,39.0%,3.2%,0.6%,0.7%,10.1%
Duwamish,1.4%,17.8%,7.5%,42.0%,0.7%,1.2%,9.7%,19.9%
Everett,2.2%,16.4%,8.7%,32.4%,1.3%,1.3%,1.9%,35.7%


Non-Work Trips

In [12]:
df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_rgc.csv')

# Create results that sum up trips for households in all RGCs
_df = df[df['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
_df['hh_rgc'] = 'All RGCs'
df = df.append(_df)

df = calculate_mode_share(df, 'hh_rgc', 'Non-Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rgc_nonwork.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rgc': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
All RGCs,2.9%,14.8%,8.1%,24.1%,1.0%,1.8%,6.6%,40.8%
Auburn,1.7%,23.1%,15.3%,29.8%,1.7%,0.9%,4.6%,22.8%
Ballard-Interbay,2.7%,18.3%,7.6%,39.2%,0.3%,0.9%,6.2%,24.9%
Bellevue,1.7%,14.7%,7.6%,24.3%,0.9%,1.3%,3.9%,45.6%
Bothell Canyon Park,2.6%,22.7%,16.7%,42.8%,1.5%,0.7%,1.2%,11.9%
Bremerton,2.2%,18.2%,10.8%,27.1%,1.5%,1.0%,1.7%,37.6%
Burien,1.8%,22.5%,14.1%,35.5%,2.6%,0.9%,5.2%,17.4%
Cascade,1.4%,27.7%,21.2%,34.3%,3.8%,0.5%,0.5%,10.7%
Duwamish,1.3%,19.9%,8.3%,38.1%,0.8%,0.8%,9.6%,21.1%
Everett,2.2%,17.7%,9.6%,28.6%,1.6%,1.3%,1.4%,37.6%


Work

In [13]:
df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_rgc.csv')

# Create results that sum up trips for households in all RGCs
_df = df[df['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
_df['hh_rgc'] = 'All RGCs'
df = df.append(_df)

df = calculate_mode_share(df, 'hh_rgc', 'Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rgc_work.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rgc': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
All RGCs,2.1%,6.5%,2.6%,34.7%,0.0%,3.4%,8.6%,42.1%
Auburn,1.1%,10.3%,4.7%,47.6%,NaN,2.2%,20.6%,13.6%
Ballard-Interbay,4.6%,8.8%,3.9%,52.6%,NaN,4.4%,9.8%,16.0%
Bellevue,1.6%,6.0%,2.6%,34.3%,NaN,3.2%,5.6%,46.8%
Bothell Canyon Park,2.0%,10.9%,7.6%,68.6%,NaN,2.0%,3.6%,5.3%
Bremerton,2.5%,8.4%,4.7%,44.3%,NaN,2.4%,6.8%,30.9%
Burien,1.8%,11.5%,4.5%,64.4%,NaN,1.8%,8.1%,7.8%
Cascade,1.9%,13.4%,5.5%,69.1%,NaN,1.4%,2.4%,6.2%
Duwamish,1.5%,8.2%,3.5%,59.3%,NaN,3.3%,9.8%,14.4%
Everett,2.1%,9.1%,3.7%,53.1%,0.0%,1.5%,4.8%,25.6%


### Regional Geography
All Trips

In [14]:
df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_rg.csv')
df = calculate_mode_share(df, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rg.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rg_proposed': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
CitiesTowns,1.0%,24.2%,18.0%,42.7%,2.5%,0.8%,1.3%,9.5%
Core,1.3%,23.4%,16.4%,42.7%,2.1%,0.9%,3.0%,10.2%
HCT,1.3%,24.1%,17.0%,43.0%,2.3%,0.9%,2.1%,9.3%
Metro,1.9%,19.8%,12.5%,37.4%,1.6%,1.5%,5.0%,20.4%
Rural,1.0%,26.3%,18.6%,45.8%,1.9%,0.6%,0.8%,5.0%
UU,1.1%,25.1%,19.2%,42.7%,2.6%,0.7%,1.0%,7.6%


Non-Work Trips

In [15]:
df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_rg.csv')
df = calculate_mode_share(df, 'hh_rg_proposed', 'Non-Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rg_nonwork.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rg_proposed': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
CitiesTowns,1.1%,26.3%,20.0%,37.9%,2.9%,0.6%,0.9%,10.2%
Core,1.3%,25.6%,18.3%,38.4%,2.5%,0.7%,2.3%,10.8%
HCT,1.3%,26.3%,19.0%,38.5%,2.6%,0.7%,1.6%,10.0%
Metro,1.9%,21.9%,14.2%,34.0%,1.9%,1.1%,4.3%,20.8%
Rural,1.1%,28.6%,20.6%,41.5%,2.2%,0.4%,0.5%,5.2%
UU,1.1%,27.3%,21.3%,37.9%,2.9%,0.6%,0.7%,8.2%


Work

In [16]:
df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_rg.csv')
df = calculate_mode_share(df, 'hh_rg_proposed', 'Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rg_work.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rg_proposed': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
CitiesTowns,0.6%,10.9%,5.3%,72.4%,0.0%,2.1%,3.8%,5.0%
Core,1.1%,10.9%,5.1%,67.5%,0.0%,2.3%,6.7%,6.4%
HCT,1.0%,11.1%,5.3%,69.9%,0.0%,2.2%,5.3%,5.3%
Metro,1.8%,9.3%,4.2%,54.6%,0.0%,3.3%,8.4%,18.3%
Rural,0.5%,10.7%,5.1%,75.7%,0.0%,1.8%,2.7%,3.5%
UU,0.7%,10.7%,5.1%,74.5%,0.0%,1.9%,3.1%,3.9%


### Equity Geography
All Trips

In [20]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_'+col+'.csv')
        df = df[df[col] == 1]
        if len(df) > 0:
            df = calculate_mode_share(df, col)
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_youth_reg,1.2%,24.5%,17.9%,42.1%,2.4%,0.9%,2.1%,9.0%
hh_elderly_reg,1.3%,23.4%,15.4%,42.9%,1.9%,1.0%,2.7%,11.4%
hh_elderly_50,0.6%,27.9%,14.3%,50.5%,1.0%,0.5%,0.8%,4.4%
hh_english_reg,1.5%,23.2%,16.5%,40.9%,2.1%,1.0%,3.4%,11.5%
hh_racial_reg,1.5%,22.7%,16.1%,39.8%,2.1%,1.1%,3.4%,13.4%
hh_racial_50,1.5%,23.2%,16.9%,39.2%,2.2%,1.0%,4.0%,12.0%
hh_poverty_reg,1.6%,22.9%,15.9%,39.4%,2.0%,1.0%,3.4%,13.9%
hh_poverty_50,2.4%,23.4%,17.6%,32.4%,2.2%,1.1%,4.3%,16.6%
hh_disability_reg,1.4%,23.3%,15.8%,41.1%,1.9%,0.9%,2.6%,12.9%


In [26]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_'+col+'.csv')
        df = df[df[col] == 0]
        if len(df) > 0:
            df = calculate_mode_share(df, col)
            df['source'] = 'NOT in '+col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
NOT in hh_youth_reg,1.6%,21.1%,13.4%,40.8%,1.6%,1.2%,3.9%,16.3%
NOT in hh_youth_50,1.4%,22.9%,15.8%,41.5%,2.0%,1.0%,2.9%,12.4%
NOT in hh_elderly_reg,1.5%,22.5%,16.2%,40.3%,2.1%,1.1%,3.1%,13.2%
NOT in hh_elderly_50,1.4%,22.9%,15.8%,41.5%,2.0%,1.0%,2.9%,12.4%
NOT in hh_english_reg,1.4%,22.8%,15.4%,41.9%,1.9%,1.0%,2.7%,13.0%
NOT in hh_english_50,1.4%,22.9%,15.8%,41.5%,2.0%,1.0%,2.9%,12.4%
NOT in hh_racial_reg,1.3%,23.1%,15.6%,42.9%,1.9%,1.0%,2.5%,11.6%
NOT in hh_racial_50,1.4%,22.8%,15.5%,42.1%,2.0%,1.0%,2.7%,12.5%
NOT in hh_poverty_reg,1.3%,23.0%,15.7%,43.0%,2.0%,1.0%,2.6%,11.3%
NOT in hh_poverty_50,1.4%,22.9%,15.8%,41.7%,2.0%,1.0%,2.9%,12.3%


Non-Work

In [21]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_'+col+'.csv')
        df = df[df[col] == 1]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Non-Work')
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_nonwork.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_youth_reg,1.3%,26.7%,19.9%,37.6%,2.8%,0.6%,1.6%,9.5%
hh_elderly_reg,1.3%,25.5%,17.2%,39.0%,2.2%,0.7%,2.2%,11.9%
hh_elderly_50,0.6%,29.5%,15.1%,48.3%,1.1%,0.3%,0.5%,4.5%
hh_english_reg,1.5%,25.3%,18.5%,36.6%,2.5%,0.8%,2.7%,12.1%
hh_racial_reg,1.5%,24.8%,18.1%,35.7%,2.4%,0.8%,2.8%,13.7%
hh_racial_50,1.6%,25.3%,18.9%,35.0%,2.6%,0.8%,3.3%,12.6%
hh_poverty_reg,1.6%,24.9%,17.8%,35.4%,2.3%,0.8%,2.8%,14.4%
hh_poverty_50,2.5%,25.2%,19.3%,28.8%,2.5%,0.9%,3.7%,17.1%
hh_disability_reg,1.4%,25.4%,17.6%,37.2%,2.2%,0.7%,2.1%,13.4%


In [27]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_'+col+'.csv')
        df = df[df[col] == 0]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Non-Work')
            df['source'] = 'NOT in '+col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_nonwork.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
NOT in hh_youth_reg,1.7%,23.3%,15.2%,37.2%,1.9%,0.9%,3.3%,16.7%
NOT in hh_youth_50,1.4%,25.1%,17.7%,37.4%,2.4%,0.8%,2.4%,12.8%
NOT in hh_elderly_reg,1.5%,24.7%,18.2%,36.0%,2.5%,0.8%,2.5%,13.6%
NOT in hh_elderly_50,1.4%,25.1%,17.7%,37.4%,2.4%,0.8%,2.4%,12.8%
NOT in hh_english_reg,1.4%,25.0%,17.3%,37.9%,2.3%,0.8%,2.1%,13.3%
NOT in hh_english_50,1.4%,25.1%,17.7%,37.4%,2.4%,0.8%,2.4%,12.8%
NOT in hh_racial_reg,1.4%,25.3%,17.5%,38.8%,2.3%,0.7%,2.0%,12.1%
NOT in hh_racial_50,1.4%,25.0%,17.4%,38.0%,2.3%,0.8%,2.1%,12.9%
NOT in hh_poverty_reg,1.3%,25.2%,17.7%,38.8%,2.4%,0.7%,2.1%,11.7%
NOT in hh_poverty_50,1.4%,25.1%,17.7%,37.6%,2.4%,0.8%,2.3%,12.7%


Work

In [22]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_'+col+'.csv')
        df = df[df[col] == 1]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Work')
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_work.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_youth_reg,0.9%,11.0%,5.3%,69.9%,0.0%,2.2%,5.2%,5.5%
hh_elderly_reg,1.1%,10.6%,4.8%,66.6%,0.0%,2.5%,6.0%,8.3%
hh_elderly_50,0.3%,10.2%,4.7%,75.3%,NaN,2.4%,3.6%,3.5%
hh_english_reg,1.2%,10.8%,5.1%,65.3%,0.0%,2.4%,7.0%,8.1%
hh_racial_reg,1.3%,10.4%,4.9%,62.7%,0.0%,2.5%,6.9%,11.3%
hh_racial_50,1.3%,10.9%,5.2%,63.3%,0.0%,2.4%,8.2%,8.7%
hh_poverty_reg,1.3%,10.6%,5.0%,62.8%,0.0%,2.3%,7.3%,10.7%
hh_poverty_50,1.6%,10.5%,5.1%,58.9%,0.0%,2.0%,9.0%,12.9%
hh_disability_reg,1.2%,10.6%,4.9%,65.3%,0.0%,2.2%,5.8%,10.0%


In [28]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(r'..\..\..\outputs\agg\dash\mode_share_'+col+'.csv')
        df = df[df[col] == 0]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Work')
            df['source'] = 'NOT in '+col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_work.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))


Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
NOT in hh_youth_reg,1.5%,9.8%,4.4%,60.0%,0.0%,2.9%,7.3%,14.1%
NOT in hh_youth_50,1.2%,10.4%,4.8%,64.9%,0.0%,2.6%,6.3%,9.9%
NOT in hh_elderly_reg,1.3%,10.2%,4.8%,63.6%,0.0%,2.6%,6.5%,11.1%
NOT in hh_elderly_50,1.2%,10.4%,4.8%,64.9%,0.0%,2.6%,6.3%,9.9%
NOT in hh_english_reg,1.2%,10.1%,4.7%,64.6%,0.0%,2.6%,5.8%,11.0%
NOT in hh_english_50,1.2%,10.4%,4.8%,64.9%,0.0%,2.6%,6.3%,9.9%
NOT in hh_racial_reg,1.2%,10.3%,4.8%,66.8%,0.0%,2.6%,5.7%,8.6%
NOT in hh_racial_50,1.2%,10.2%,4.7%,65.3%,0.0%,2.6%,5.8%,10.2%
NOT in hh_poverty_reg,1.1%,10.2%,4.7%,66.3%,0.0%,2.7%,5.6%,9.3%
NOT in hh_poverty_50,1.2%,10.4%,4.8%,65.0%,0.0%,2.6%,6.2%,9.8%
